Darren Colby

MSCA 32020

Homework 4

Partners: Shogo Nakano

C is the cost of the car at age i, T is the trade-in value of the car at age i, E is the cost of ownership at age i, and P is the <br>
probability of the car surviving to age i.

In [55]:
import numpy as np
import pandas as pd

In [56]:
replacement = pd.read_csv("./howard_autoReplacement_data.csv")
replacement.head()

,i,C,T,E,p
0,0,2000,1600,50,1.000
1,1,1840,1460,53,0.999
2,2,1680,1340,56,0.998
3,3,1560,1230,59,0.997
4,4,1300,1050,62,0.996


In [57]:
actions = np.arange(1, 42, 1)

## Transition Function

The code below creates the transition probability function, which in this case, is a tensor

In [58]:
P = np.zeros((len(actions), 40, 40))
_, s, _ = P.shape

for a in actions:
    P_mat = np.zeros((40, 40))
    
    # If a == 1, keep the car
    if a == 1:
        for i in range(s):
            for j in range(s):
                if i+1 == j:
                    P_mat[i, j] = replacement['p'][i +  1]
                if j == s - 1:
                    P_mat[i, j] = 1 - replacement['p'][i + 1]
        P_mat[38, 39] = 1
    
    # If a is 41, the probability is 1
    elif a == 41:
        for i in range(s):
            P_mat[i, -1] = 1
    
    # For other actions,
    else:
        for i in range(s):
            P_mat[i, a - 2] = replacement['p'][a - 2]
            P_mat[i, -1] = 1 - replacement['p'][a - 2]
            
    P[a - 1] = P_mat

## Reward Function

If we keep the car, the reward is the cost of maintenance. If we sell the car, the reward is the trad-in cost less the cost of the new car and the maintenance cost.

In [59]:
# Q matrix
q = np.zeros((41, 40, 1), dtype=np.float64)

# There are N states and K rewards
K, N, M = P.shape

# reward matrix
for i in range(N):
    for k in range(K):
        if k == 0:
            q[k, i, 0] = -replacement['E'][i + 1]
        else:
            q[k,i,0] = replacement['T'][i + 1] - replacement['C'][k - 1] - replacement['E'][k - 1]

## Policy Iteration

In [60]:
T=8

# Values and decisions
v, d = np.zeros((N, T), dtype=np.float64), np.ones((N, T), dtype=int)

# Auxillary variables
PP, qvec = np.zeros((N, N), dtype=np.float64), np.zeros((N, 1), dtype=np.float64)

for n in range(1,T):

    # Policy improvement
    for i in range(N):
        rhs = np.zeros((1, K),dtype=np.float64)
        for k in range(K):
            rhs[0][k] = q[k][i][0] + np.matmul(P[k, i, :], v[:, n - 1]) 
        v[i, n] = max(rhs[0])
        d[i, n] = np.argmax(rhs[0])

    # Value determination
    for i in range(0,N):
        PP[i, :] = P[d[i, n], i,:]
    
    A = np.concatenate((np.identity(N) - PP, np.ones((N, 1))), axis=1)
    A = np.delete(A, N - 1, 1)
    
    for i in range(N):
        qvec[i, 0] = q[d[i, n], i, 0]

    tmp = np.matmul(np.linalg.inv(A), qvec)
    g = tmp[N - 1]
    tmp[N - 1] = 0
    v[:, n] = tmp.T

v = v.T
d = d.T

## Determine the Best Policy

In [62]:
decision = np.where(d[-1] == 0, 'K', d[-1] - 1)
pd.DataFrame({'State': range(1, 41), 'Action': decision, 'Value': v[-1]})

,State,Action,Value
0,1,12,1380.000000
1,2,12,1260.000000
2,3,K,1160.661162
3,4,K,1071.931119
4,5,K,986.933014
5,6,K,906.425732
6,7,K,830.958523
7,8,K,760.134298
8,9,K,694.607575
9,10,K,632.412756
